# Minimum Energy Conical Intersections

In this module we show how pyGSM works with pyTC (Lightspeed) to optimize minimum energy conical intersections.

In [1]:
import sys
sys.path.insert(0,'/home/caldaz/module/pyGSM')
from molecule import Molecule
from pes import PES
from avg_pes import Avg_PES
import numpy as np
from nifty import pvec1d,pmat2d
import matplotlib
import matplotlib.pyplot as plt
from pytc import *
import manage_xyz
from rhf_lot import *
from psiw import *
from nifty import getAllCoords,getAtomicSymbols,click,printcool
import pybel as pb
%matplotlib inline

In [2]:
printcool("Build resources")
resources = ls.ResourceList.build()
printcool('{}'.format(resources))

#========================================================#
#|                   Build resources                    |#
#========================================================#
#==============================================================#
#|                       ResourceList:                        |#
#|                       CPU threads:  16                     |#
#|                       GPU contexts:  1                     |#
#|                     GPU context details:                   |#
#|    N  ID                 Name  CC  CPU Buffer  GPU Buffer  |#
#|    0   0    GeForce GTX TITAN 3.5        1024        1024  |#
#|                                                            |#
#==============================================================#


'----------------------------------------------------------------\n'

In [3]:
printcool("build the Lightspeed (pyTC) objecs")

filepath='data/twisted_fluoroethene.xyz'
molecule = ls.Molecule.from_xyz_file(filepath)
geom = geometry.Geometry.build(
    resources=resources,
    molecule=molecule,
    basisname='6-31gs',
    )
printcool('{}'.format(geom))

ref = RHF.from_options(
     geometry= geom, 
     g_convergence=1.0E-6,
     fomo=True,
     fomo_method='gaussian',
     fomo_temp=0.3,
     fomo_nocc=11,
     fomo_nact=2,
     print_level=1,
    )
ref.compute_energy()
casci = CASCI.from_options(
    reference=ref,
    nocc=11,
    nact=2,
    nalpha=1,
    nbeta=1,
    S_inds=[0],
    S_nstates=[2],
    print_level=1,
    )
casci.compute_energy()
psiw = CASCI_LOT.from_options(
    casci=casci,
    rhf_guess=True,
    rhf_mom=True,
    orbital_coincidence='core',
    state_coincidence='full',
    )

#========================================================#
#|          build the Lightspeed (pyTC) objecs          |#
#========================================================#
#========================================================#
#|                      Geometry:                       |#
#|                      QMMM = False                    |#
#|                      -D3  = False                    |#
#|                      ECP  = False                    |#
#|                                                      |#
#|            Molecule: twisted_fluoroethene            |#
#|               Natom        =           6             |#
#|               Charge       =       0.000             |#
#|               Multiplicity =       1.000             |#
#|                                                      |#
#|                    Basis: 6-31gs                     |#
#|                    nao     =    51                   |#
#|                    ncart   =    51                   

In [4]:
printcool("Build the pyGSM Level of Theory object (LOT)")
lot=PyTC.from_options(states=[(1,0),(1,1)],psiw=psiw,do_coupling=True,fnm=filepath)

#========================================================#
#|     Build the pyGSM Level of Theory object (LOT)     |#
#========================================================#
 initializing LOT from file


In [5]:
printcool("Build the pyGSM Potential Energy Surface Object (PES)")
pes1 = PES.from_options(lot=lot,ad_idx=0,multiplicity=1)
pes2 = PES.from_options(lot=lot,ad_idx=1,multiplicity=1)
pes = Avg_PES(pes1,pes2,lot=lot)

#===========================================================#
#|  Build the pyGSM Potential Energy Surface Object (PES)  |#
#===========================================================#


In [6]:
printcool("Build the pyGSM Molecule object \n with Translation and Rotation Internal Coordinates (TRIC)")
M = Molecule.from_options(fnm=filepath,PES=pes,coordinate_type="TRIC")

#================================================================#
#|               Build the pyGSM Molecule object                |#
#|   with Translation and Rotation Internal Coordinates (TRIC)  |#
#================================================================#
 reading cartesian coordinates from file
 making primitives from options!
 making primitive Hessian
 forming Hessian in basis


In [7]:
from eigenvector_follow import eigenvector_follow
ef = eigenvector_follow.from_options() #Linesearch=NoLineSearch)

In [8]:
geoms,E = ef.optimize(
    molecule=M,
    refE=M.energy,
    opt_type="MECI",
    opt_steps=50,
)

initial E 0.0000
MECI
 On opt step 1 
 actual_step= 0.20
 reducing step, new step = 0.10
 pgradrms [[ 0.1705]]
 norm_dg is 0.2040
 dE is 108.6759
[[-0.075  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.   ]]
 ### Starting  line search ###
 [INFO]end line evaluate fx = -110875.6816 step = 0.10.
 ## Done line search
[[-0.075   0.      0.0079  0.0291 -0.0105  0.0006 -0.0056  0.0384 -0.011
   0.007  -0.0384  0.0275 -0.0328  0.0384  0.0182  0.0384 -0.0228 -0.0143]]
 DMAX 0.10
 Opt step: 1 E: -23.4417 predE: -31.8208 ratio: 0.737 gradrms: 0.09493 ss: 0.100 DMAX: 0.100

MECI
updating coordinate basis
('dE is ', 98.9942653032922)
 On opt step 2 
 actual_step= 0.17
 reducing step, new step = 0.10
 pgradrms [[ 0.0977]]
 norm_dg is 0.2106
 dE is 98.9943
[[-0.075  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.   ]]
 ### Starting  line search ###
 [INFO]end line eval